In [1]:
val path = java.nio.file.FileSystems.getDefault().getPath("/Users/pauldanilin/Documents/HSE/CourseWorks/HistogramModel/target/scala-2.11/histogrammodel_2.11-0.1.jar")
val x = ammonite.ops.Path(path)
interp.load.cp(x)

path: java.nio.file.Path = /Users/pauldanilin/Documents/HSE/CourseWorks/HistogramModel/target/scala-2.11/histogrammodel_2.11-0.1.jar
x: ammonite.ops.Path = /Users/pauldanilin/Documents/HSE/CourseWorks/HistogramModel/target/scala-2.11/histogrammodel_2.11-0.1.jar

In [2]:
import ru.hse.se.ba.danilin.paul.histogram_model.histogram.Implicits._
import ru.hse.se.ba.danilin.paul.histogram_model.histogram.Histogram

import $ivy.`org.plotly-scala::plotly-jupyter-scala:0.3.1`

import plotly._
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._

plotly.JupyterScala.init()

import ru.hse.se.ba.danilin.paul.histogram_model.histogram.Implicits._

import ru.hse.se.ba.danilin.paul.histogram_model.histogram.Histogram


import $ivy.$                                             


import plotly._

import plotly.element._

import plotly.layout._

import plotly.JupyterScala._



In [3]:
implicit class DrawableHistogram[E](hist: Histogram[E]) {
    def draw() = {
        val xs = hist.elementsPresent.toSeq
        val ys = xs.map(e => hist(e).toInt)
        Bar(xs.map(_.toString), ys).plot()
    }
}

defined class DrawableHistogram

In [6]:
val image = javax.imageio.ImageIO.read(new java.io.File("/Users/pauldanilin/Desktop/test_image.jpg"))

image: java.awt.image.BufferedImage = BufferedImage@657dd465: type = 5 ColorModel: #pixelBits = 24 numComponents = 3 color space = java.awt.color.ICC_ColorSpace@5f2896e9 transparency = 1 has alpha = false isAlphaPre = false ByteInterleavedRaster: width = 1140 height = 550 #numDataElements 3 dataOff[0] = 2

In [7]:
val v = image.toHistogram

v: Histogram[ru.hse.se.ba.danilin.paul.histogram_model.atomizers.Pixel] = HistogramImpl(Map(GREEN -> 538695.0, RED -> 57451.0, BLUE -> 30854.0))

In [9]:
v.draw

res8: String = "plot-1989874599"